<h1>Gaussian Mixture Model</h1>

# 0. EM

# 0.0 EM推导

参数估计首先想到极大似然估计。

假设模型的观测数据$Y$，隐变量（不可观测数据）$Z$，参数为$\theta$，完全数据$(Y, Z)$的联合概率分布$P(Y, Z|\theta)$。要估计参数$\theta$，首先计算观测数据关于参数的log似然。

> $L(\theta|Y) = \log p(Y|\theta)$

考虑完全数据的log似然，
> $\log p(Y, Z|\theta) = \log p(Y|\theta) p(Z|Y, \theta)
= \log p(Y|\theta) + \log p(Z|Y, \theta)$

从而$Y$的log似然，

> $\log p(Y|\theta) = \log p(Y,Z|\theta) - \log p(Z|Y, \theta)$

引入$Z$的另一个分布$q(Z)$，两边同时乘以$\sum_{Z} q(Z)$，

> $\sum_{Z} q(Z) \log P(Y|\theta) = \sum_{Z} q(Z) \left(\log p(Y,Z|\theta) - \log p(Z|Y, \theta)\right)$

> $\log p(Y|\theta) = \sum_{Z} q(Z) \left( \log p(Y,Z|\theta) - \log q(Z) + \log q(Z) - \log p(Z|Y, \theta) \right)$

> $ = \sum_Z q(Z) \left( \log \frac{p(Y,Z|\theta)} {q(Z)} -  
\frac{\log p(Z|Y, \theta)}{q(Z)} \right)$

> $ = \sum_Z q(Z) \log \frac{p(Y,Z|\theta)} {q(Z)} - \sum_Z q(Z) \frac{\log p(Z|Y, \theta)}{q(Z)} $

> $ = \mathcal{L}(q, \theta) + \mathbb{KL}[q||p]$

> $\mathcal{L}(q, \theta) = \sum_Z q(Z) \log \frac{p(Y,Z|\theta)} {q(Z)} 
= \mathbb{E}_q [\log p(Y,Z|\theta)]$

> $\mathbb{KL}[q||p] =  - \sum_Z q(Z) \frac{\log p(Z|Y, \theta)}{q(Z)}$

考虑不完全$Y$的数据的log似然，从完全数据$(Y,Z)$的log似然分解开始，然后引入隐变量的另一个分布$q(Z)$。

> $\log p(Y|\theta) = \mathcal{L}(q, \theta) + \mathbb{KL}[q||p]$

> $ >= \mathcal{L}(q, \theta)$

$\mathcal{L}(q, \theta)$是$\log p(Y|\theta)$的下界，当$\mathbb{KL}[q||p] = 0$时，$\mathcal{L}(q, \theta)$取得最大值，等于$\log p(Y|\theta)$，此时$q(Z) = p(Z|Y,\theta)$。

## 0.1 EM步骤

EM算法是迭代算法，分E-步与M-步。

+ E-步：根据$\theta^{old}$计算完全数据关于隐变量的期望$\mathbb{E}_{p(Z|Y, \theta^{old})} [\log p(Y,Z|\theta)]$

当$q(Z) = p(Z|Y, \theta^{old})$，

> $\mathcal{L}(q, \theta) = \sum_Z  p(Z|Y, \theta^{old}) \log \frac{p(Y,Z|\theta)} { p(Z|Y, \theta^{old})} 
 = \mathbb{E}_{p(Z|Y, \theta^{old})} [\log \frac{p(Y,Z|\theta)} { p(Z|Y, \theta^{old})}]$

需要计算$p(Z|Y, \theta^{old})$，根据贝叶斯规则，

> $p(Z|Y, \theta^{old}) = 
\frac{p(Y|Z, \theta^{old})p(Z|\theta)}
{\sum_Z p(Y|Z, \theta^{old})p(Z|\theta)}$

 
+ M-步：最大化$\mathbb{E}_{p(Z|Y, \theta^{old})} [\log \frac{p(Y,Z|\theta)} { p(Z|Y, \theta^{old})}] $，计算得到$\theta^{new}$

> $\theta^{new} = \arg\max_{\theta} \mathbb{E}_{p(Z|Y, \theta^{old})} [\log \frac{p(Y,Z|\theta)} { p(Z|Y, \theta^{old})}] $

> $ = \arg\max_{\theta} \mathbb{E}_{p(Z|Y, \theta^{old})} [\log p(Y,Z|\theta) ]$

此时，$q(Z) = p(Z|Y, \theta^{new})$。

所以需要在E-步计算$\mathbb{E}_{p(Z|Y, \theta^{old})} [\log p(Y,Z|\theta)]$。

重复E-步与M-步，直到满足停止迭代条件。